### Master Thesis: EigenGARCH(2021)
We consider the new multivariate Dynamic Conditional Eigenvalue GARCH model (λ-MGARCH) introduced in Hetland et al. (2022 JoE). We are interested in evaluating its ability to forecast conditional covariance matrices, relative to competing multivariate GARCH models for time- varying conditional covariance matrice

### Initial Estimation

In [24]:
# Import Packages
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import gamma
from scipy.stats import stats
from matplotlib import pyplot as plt 
import statsmodels.api as sm
import statsmodels as sms
import yfinance as yf
from numpy.linalg import eig

import Eigen_GARCH as Eigen_GARCH

#### Import Data

In [25]:
# Use the exact same data as in R, for the DCC and GO-GARCH model
data = np.array(pd.read_csv("returns_R.csv")).T

#### Estimate

In [26]:
# Set up bounds and lambdas
covdata = np.cov(data,bias=True)
T = data.shape[1]
k = 3
eigenvalues, eigenvectors = eig(covdata)
lambdas_eig = eigenvalues
bnds = [(-np.pi/2, np.pi/2)]
bnds_eig = [(0, np.pi)]

for i in range(2*(k**2)+k+2):
    bnds_eig.append((0, 100))

# Initial values
arguments_eig = (np.asarray(data),lambdas_eig)
startingVals_eig = [np.random.uniform(0.01,0.2) for i in range(2*(k**2) + k + 3)]

# Estimation
estimation = minimize(Eigen_GARCH.eigenLikelihood , x0=startingVals_eig , 
                          args=arguments_eig , method="SLSQP", bounds=bnds_eig, 
                          options={"maxiter":100000, "ftol":10e-14})

/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:39: RuntimeWarning: overflow encountered in matmul
  lambdas[:,t:t+1] = W + A @ np.multiply(Xtilde[:,t-1:t],Xtilde[:,t-1:t]) + B @ lambdas[:,t-1:t]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:44: RuntimeWarning: divide by zero encountered in log
  lls3 = np.log(lambdas[2,:]) + np.multiply(Xtilde[2,:],Xtilde[2,:]) / lambdas[2,:]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:44: RuntimeWarning: divide by zero encountered in true_divide
  lls3 = np.log(lambdas[2,:]) + np.multiply(Xtilde[2,:],Xtilde[2,:]) / lambdas[2,:]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:44: RuntimeWarning: invalid value encountered in true_divide
  lls3 = np.log(lambdas[2,:]) + np.multiply(Xtilde[2,:],Xtilde[2,:]) / lambdas[2,:]
/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:44: RuntimeWarning: invalid value encountered in add
  lls3 = np.log(lambdas[2,:]) + np.multiply(Xtilde[2,:],Xtilde[2,:]) / lambdas[2,:]
/Users/silkesommerfug

In [27]:
# Save the estimated parameter values
phi1_eig, phi2_eig, phi3_eig, w1_eig, w2_eig, w3_eig , a11_eig , a12_eig , a13_eig, a21_eig , a22_eig, a23_eig, a31_eig , a32_eig, a33_eig , b11_eig , b12_eig, b13_eig , b21_eig , b22_eig, b23_eig, b31_eig , b32_eig, b33_eig = estimation.x

# Find the llh and information criteria for the Eigen GARCH
llh = - estimation.fun
AIC = 2 * 24 - 2 * (llh)
BIC = - 2 * llh + np.log(T) * 24

# Load into matrices
W_eig = np.array(([w1_eig, w2_eig, w3_eig]))
A_eig = np.array(([a11_eig, a12_eig, a13_eig],[a21_eig, a22_eig, a23_eig],[a31_eig, a32_eig, a33_eig]))
B_eig = np.array(([b11_eig, b12_eig, b13_eig],[b21_eig, b22_eig, b23_eig],[b31_eig, b32_eig, b33_eig]))
V_eig = np.array(([np.cos(phi1_eig), np.sin(phi1_eig),0], [-np.sin(phi1_eig), np.cos(phi1_eig),0 ], [0, 0,1 ]))@ np.array(([np.cos(phi2_eig), 0, np.sin(phi2_eig)], [0, 1,0 ], [-np.sin(phi2_eig), 0, np.cos(phi2_eig)]))@ np.array(( [1, 0,0 ], [0,np.cos(phi3_eig), np.sin(phi3_eig)], [0,-np.sin(phi3_eig), np.cos(phi3_eig)]))

print(f'W: {W_eig}')
print(f'A: {A_eig}')
print(f'B: {B_eig}')
print(f'V: {V_eig}')
print(f'Log-likelihood: {llh}')
print(f'AIC: {AIC}')
print(f'BIC: {BIC}')

W: [7.22861260e-03 1.07086466e-07 4.39627225e-02]
A: [[0.0850894  0.00948949 0.01576432]
 [0.09565499 0.04868495 0.02530218]
 [0.00503486 0.02105131 0.06244392]]
B: [[9.05831491e-01 1.04782494e-13 7.48321813e-08]
 [5.45065093e-03 4.74779313e-01 7.41143461e-01]
 [3.66812624e-08 4.36060944e-13 8.44307123e-01]]
V: [[ 1.00000000e+00  1.03283498e-12  2.91355365e-12]
 [-1.03283498e-12  1.00000000e+00  2.02021973e-12]
 [-2.91355365e-12 -2.02021973e-12  1.00000000e+00]]
Log-likelihood: -12842.929860497505
AIC: 25733.85972099501
BIC: 25871.22522032968


In [28]:
# Residuals
residuals = Eigen_GARCH.eigenresiduals(estimation.x, np.asarray(data), lambdas_eig)

In [29]:
# Misspisification test
print(stats.jarque_bera(residuals[0]))
print(stats.jarque_bera(residuals[1]))
print(stats.jarque_bera(residuals[2]))

print(sm.stats.acorr_ljungbox(residuals[0], lags=[10], return_df=True))
print(sm.stats.acorr_ljungbox(residuals[1], lags=[10], return_df=True))
print(sm.stats.acorr_ljungbox(residuals[2], lags=[10], return_df=True))

Jarque_beraResult(statistic=177468.97363991258, pvalue=0.0)
Jarque_beraResult(statistic=193786.85759486962, pvalue=0.0)
Jarque_beraResult(statistic=365662.8208187752, pvalue=0.0)
      lb_stat     lb_pvalue
10  79.380086  6.640464e-13
       lb_stat     lb_pvalue
10  187.604126  6.160778e-35
      lb_stat     lb_pvalue
10  97.283924  1.902540e-16


In [30]:
# Calculate the covariance matrices based on the estimated values
Xtilde = V_eig.transpose() @ data
lambdas = (np.ones((k,T))).transpose()
condVar_eig = np.zeros((T,k,k))

for t in range(1,T):
    lambdas[t] = W_eig+np.concatenate(A_eig @ np.multiply(Xtilde[:,t-1:t],Xtilde[:,t-1:t]), axis=0 )+ B_eig @ lambdas[t-1]
    condVar_eig[t] = V_eig @ np.diag(lambdas[t]) @ V_eig.T

# Make a list for each of the covariance pairs
cov_JPM_KO = []
for t in range(0,T):
    cov_JPM_KO.append(condVar_eig[t][0][1])

cov_JPM_MSFT = []
for t in range(0,T):
    cov_JPM_MSFT.append(condVar_eig[t][0][2])

cov_KO_MSFT = []
for t in range(0,T):
    cov_KO_MSFT.append(condVar_eig[t][1][2])


In [31]:
# Save results 
df = pd.DataFrame(cov_JPM_KO)
df.to_csv('cov_JPM_KO_eigen.csv') 

df = pd.DataFrame(cov_JPM_MSFT)
df.to_csv('cov_JPM_MSFT_eigen.csv') 

df = pd.DataFrame(cov_KO_MSFT)
df.to_csv('cov_KO_MSFT_eigen.csv')


### Pre period

In [64]:
# Use the exact same data as in R, for the DCC and GO-GARCH model
data_pre = np.array(pd.read_csv("returns_R.csv")[0:1506]).T

In [65]:
# Set up bounds and lambdas
covdata = np.cov(data_pre,bias=True)
T = data_pre.shape[1]
k = 3
eigenvalues, eigenvectors = eig(covdata)
lambdas_eig = eigenvalues
bnds = [(-np.pi/2, np.pi/2)]
bnds_eig = [(0, np.pi)]

for i in range(2*(k**2)+k+2):
    bnds_eig.append((0, 100))

# Initial values
arguments_eig = (np.asarray(data_pre),lambdas_eig)
startingVals_eig = [np.random.uniform(0.01,0.2) for i in range(2*(k**2) + k + 3)]

# Estimation
estimation = minimize(Eigen_GARCH.eigenLikelihood , x0=startingVals_eig , 
                          args=arguments_eig , method="SLSQP", bounds=bnds_eig, 
                          options={"maxiter":100000, "ftol":10e-14})

/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:42: RuntimeWarning: overflow encountered in true_divide
  lls1 = np.log(lambdas[0,:]) + np.multiply(Xtilde[0,:],Xtilde[0,:]) / lambdas[0,:]


In [66]:
# Save the estimated parameter values
phi1_eig, phi2_eig, phi3_eig, w1_eig, w2_eig, w3_eig , a11_eig , a12_eig , a13_eig, a21_eig , a22_eig, a23_eig, a31_eig , a32_eig, a33_eig , b11_eig , b12_eig, b13_eig , b21_eig , b22_eig, b23_eig, b31_eig , b32_eig, b33_eig = estimation.x

# Find the llh and information criteria for the Eigen GARCH
llh = - estimation.fun
AIC = 2 * 24 - 2 * (llh)
BIC = - 2 * llh + np.log(T) * 24

# Load into matrices
W_eig = np.array(([w1_eig, w2_eig, w3_eig]))
A_eig = np.array(([a11_eig, a12_eig, a13_eig],[a21_eig, a22_eig, a23_eig],[a31_eig, a32_eig, a33_eig]))
B_eig = np.array(([b11_eig, b12_eig, b13_eig],[b21_eig, b22_eig, b23_eig],[b31_eig, b32_eig, b33_eig]))
V_eig = np.array(([np.cos(phi1_eig), np.sin(phi1_eig),0], [-np.sin(phi1_eig), np.cos(phi1_eig),0 ], [0, 0,1 ]))@ np.array(([np.cos(phi2_eig), 0, np.sin(phi2_eig)], [0, 1,0 ], [-np.sin(phi2_eig), 0, np.cos(phi2_eig)]))@ np.array(( [1, 0,0 ], [0,np.cos(phi3_eig), np.sin(phi3_eig)], [0,-np.sin(phi3_eig), np.cos(phi3_eig)]))

print(f'Log-likelihood: {llh}')
print(f'AIC: {AIC}')
print(f'BIC: {BIC}')

Log-likelihood: -7841.856583912293
AIC: 15731.713167824586
BIC: 15859.326265625223


### Post period

In [67]:
# Use the exact same data as in R, for the DCC and GO-GARCH model
data_post = np.array(pd.read_csv("returns_R.csv")[1506:]).T

In [68]:
# Set up bounds and lambdas
covdata = np.cov(data_post,bias=True)
T = data_post.shape[1]
k = 3
eigenvalues, eigenvectors = eig(covdata)
lambdas_eig = eigenvalues
bnds = [(-np.pi/2, np.pi/2)]
bnds_eig = [(0, np.pi)]

for i in range(2*(k**2)+k+2):
    bnds_eig.append((0, 100))

# Initial values
arguments_eig = (np.asarray(data_post),lambdas_eig)
startingVals_eig = [np.random.uniform(0.01,0.2) for i in range(2*(k**2) + k + 3)]

# Estimation
estimation = minimize(Eigen_GARCH.eigenLikelihood , x0=startingVals_eig , 
                          args=arguments_eig , method="SLSQP", bounds=bnds_eig, 
                          options={"maxiter":100000, "ftol":10e-14})

In [69]:
# Save the estimated parameter values
phi1_eig, phi2_eig, phi3_eig, w1_eig, w2_eig, w3_eig , a11_eig , a12_eig , a13_eig, a21_eig , a22_eig, a23_eig, a31_eig , a32_eig, a33_eig , b11_eig , b12_eig, b13_eig , b21_eig , b22_eig, b23_eig, b31_eig , b32_eig, b33_eig = estimation.x

# Find the llh and information criteria for the Eigen GARCH
llh = - estimation.fun
AIC = 2 * 24 - 2 * (llh)
BIC = - 2 * llh + np.log(T) * 24

# Load into matrices
W_eig = np.array(([w1_eig, w2_eig, w3_eig]))
A_eig = np.array(([a11_eig, a12_eig, a13_eig],[a21_eig, a22_eig, a23_eig],[a31_eig, a32_eig, a33_eig]))
B_eig = np.array(([b11_eig, b12_eig, b13_eig],[b21_eig, b22_eig, b23_eig],[b31_eig, b32_eig, b33_eig]))
V_eig = np.array(([np.cos(phi1_eig), np.sin(phi1_eig),0], [-np.sin(phi1_eig), np.cos(phi1_eig),0 ], [0, 0,1 ]))@ np.array(([np.cos(phi2_eig), 0, np.sin(phi2_eig)], [0, 1,0 ], [-np.sin(phi2_eig), 0, np.cos(phi2_eig)]))@ np.array(( [1, 0,0 ], [0,np.cos(phi3_eig), np.sin(phi3_eig)], [0,-np.sin(phi3_eig), np.cos(phi3_eig)]))

print(f'Log-likelihood: {llh}')
print(f'AIC: {AIC}')
print(f'BIC: {BIC}')

Log-likelihood: -4698.375843814547
AIC: 9444.751687629094
BIC: 9555.79291361107


### Forecast

In [63]:
# Use the exact same data as in R, for the DCC and GO-GARCH model
data_forecast = data

In [28]:
# Number of loops in forecast
p = 3
N = 755
# Create parameter containers and covariance matrix container
phi1_eig = np.zeros((N,1))
phi2_eig = np.zeros((N,1))
phi3_eig = np.zeros((N,1))

w1_eig = np.zeros((N,1))
w2_eig = np.zeros((N,1))
w3_eig = np.zeros((N,1))

a11_eig = np.zeros((N,1))
a12_eig = np.zeros((N,1))
a13_eig = np.zeros((N,1))
a21_eig = np.zeros((N,1))
a22_eig = np.zeros((N,1))
a23_eig = np.zeros((N,1))
a31_eig = np.zeros((N,1))
a32_eig = np.zeros((N,1))
a33_eig = np.zeros((N,1))

b11_eig = np.zeros((N,1))
b12_eig = np.zeros((N,1))
b13_eig = np.zeros((N,1))
b21_eig = np.zeros((N,1))
b22_eig = np.zeros((N,1))
b23_eig = np.zeros((N,1))
b31_eig = np.zeros((N,1))
b32_eig = np.zeros((N,1))
b33_eig = np.zeros((N,1))

forecastMatrix_eig = np.zeros((N,p,p))

In [29]:
for n in range(N):
    # Move estimation window by 5
    X = data_forecast[:,n:1486+n]
    X_f = data_forecast[:,1486+n:1486+n+1]
    print(n, "out of", N, "in Eigen-GARCH forecasting")
    # Generate new lambdas for likelihood estimation
    lambdas_eig = np.tile(np.linalg.eig(np.cov(X.T, rowvar=False))[0], (T,1))
    
    # Use prior estimation results as starting values
    startingVals_eig = list(estimation.x)
    
    # Arguments
    arguments_eig = (np.asarray(X), lambdas_eig)
    
    # Estimation
    estimation = minimize(Eigen_GARCH.eigenLikelihood, x0=startingVals_eig , args=arguments_eig , method="SLSQP", bounds=bnds_eig, options={"maxiter":100000, "ftol":10e-24})
    
    # Save parameters
    phi1_eig[n], phi2_eig[n], phi3_eig[n], w1_eig[n], w2_eig[n], w3_eig[n] , a11_eig[n] , a12_eig[n] , a13_eig[n], a21_eig [n], a22_eig[n], a23_eig[n], a31_eig[n] , a32_eig[n], a33_eig[n] , b11_eig[n] , b12_eig[n], b13_eig[n] , b21_eig[n] , b22_eig[n], b23_eig[n], b31_eig[n] , b32_eig[n], b33_eig[n] = estimation.x
    phi1, phi2, phi3, w1, w2, w3, a11, a12, a13, a21, a22, a23, a31, a32, a33, b11, b12, b13, b21, b22,b23, b31, b32,b33 = estimation.x
 
    W_eig = np.array(([w1, w2, w3]))
    A_eig = np.array(([a11, a12,a13],
                  [a21, a22,a23],
                  [a31, a32,a33]))
    B_eig = np.array(([b11, b12,b13],
                  [b21, b22,b23],
                  [b31, b32,b33]))
    V_eig = np.array(([np.cos(phi1), np.sin(phi1),0],[-np.sin(phi1), np.cos(phi1),0 ], [0, 0,1 ])
                    )@ np.array(([np.cos(phi2), 0, np.sin(phi2)], [0, 1,0 ], [-np.sin(phi2), 0, np.cos(phi2)])
                           )@ np.array(( [1, 0,0 ], [0,np.cos(phi3), np.sin(phi3)], [0,-np.sin(phi3), np.cos(phi3)]))
    
    Xtilde = V_eig.transpose() @ X
    T = 1486
    lambdas = (np.ones((k,T))).transpose()
    
    condVar_eig = np.zeros((T,k,k))
    
    for t in range(1,T):
        lambdas[t] = W_eig+np.concatenate(A_eig @ np.multiply(Xtilde[:,t-1:t],Xtilde[:,t-1:t]), axis=0 )+ B_eig @ lambdas[t-1]
        condVar_eig[t] = V_eig @ np.diag(lambdas[t]) @ V_eig.T
     
    Xtilde_f = V_eig.transpose() @ X_f
    
    lambdas_f = W_eig + np.concatenate(A_eig @ np.multiply(Xtilde_f,Xtilde_f), axis=0 )+ B_eig @ lambdas[T-1]
 
    # Conditional eigenvalue at time t+1
    #Xtilde = V_eig.transpose() @ X
    
    # Store 1-step forecast
    #forecastMatrix_eig[n,:,:] = condVar_eig[T-1]
    forecastMatrix_eig[n,:,:] = V_eig @ np.diag(lambdas_f) @ V_eig.T


0 out of 754 in Eigen-GARCH forecasting
1 out of 754 in Eigen-GARCH forecasting
2 out of 754 in Eigen-GARCH forecasting
3 out of 754 in Eigen-GARCH forecasting
4 out of 754 in Eigen-GARCH forecasting
5 out of 754 in Eigen-GARCH forecasting
6 out of 754 in Eigen-GARCH forecasting
7 out of 754 in Eigen-GARCH forecasting
8 out of 754 in Eigen-GARCH forecasting
9 out of 754 in Eigen-GARCH forecasting
10 out of 754 in Eigen-GARCH forecasting
11 out of 754 in Eigen-GARCH forecasting
12 out of 754 in Eigen-GARCH forecasting
13 out of 754 in Eigen-GARCH forecasting
14 out of 754 in Eigen-GARCH forecasting
15 out of 754 in Eigen-GARCH forecasting
16 out of 754 in Eigen-GARCH forecasting
17 out of 754 in Eigen-GARCH forecasting
18 out of 754 in Eigen-GARCH forecasting


/Users/silkesommerfugl/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


19 out of 754 in Eigen-GARCH forecasting
20 out of 754 in Eigen-GARCH forecasting
21 out of 754 in Eigen-GARCH forecasting
22 out of 754 in Eigen-GARCH forecasting
23 out of 754 in Eigen-GARCH forecasting
24 out of 754 in Eigen-GARCH forecasting
25 out of 754 in Eigen-GARCH forecasting
26 out of 754 in Eigen-GARCH forecasting
27 out of 754 in Eigen-GARCH forecasting
28 out of 754 in Eigen-GARCH forecasting
29 out of 754 in Eigen-GARCH forecasting
30 out of 754 in Eigen-GARCH forecasting
31 out of 754 in Eigen-GARCH forecasting
32 out of 754 in Eigen-GARCH forecasting
33 out of 754 in Eigen-GARCH forecasting
34 out of 754 in Eigen-GARCH forecasting
35 out of 754 in Eigen-GARCH forecasting
36 out of 754 in Eigen-GARCH forecasting
37 out of 754 in Eigen-GARCH forecasting
38 out of 754 in Eigen-GARCH forecasting
39 out of 754 in Eigen-GARCH forecasting
40 out of 754 in Eigen-GARCH forecasting
41 out of 754 in Eigen-GARCH forecasting
42 out of 754 in Eigen-GARCH forecasting
43 out of 754 in

/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:44: RuntimeWarning: invalid value encountered in true_divide
  lls3 = np.log(lambdas[2,:]) + np.multiply(Xtilde[2,:],Xtilde[2,:]) / lambdas[2,:]


134 out of 754 in Eigen-GARCH forecasting
135 out of 754 in Eigen-GARCH forecasting
136 out of 754 in Eigen-GARCH forecasting
137 out of 754 in Eigen-GARCH forecasting
138 out of 754 in Eigen-GARCH forecasting
139 out of 754 in Eigen-GARCH forecasting
140 out of 754 in Eigen-GARCH forecasting
141 out of 754 in Eigen-GARCH forecasting
142 out of 754 in Eigen-GARCH forecasting
143 out of 754 in Eigen-GARCH forecasting
144 out of 754 in Eigen-GARCH forecasting
145 out of 754 in Eigen-GARCH forecasting
146 out of 754 in Eigen-GARCH forecasting
147 out of 754 in Eigen-GARCH forecasting
148 out of 754 in Eigen-GARCH forecasting
149 out of 754 in Eigen-GARCH forecasting
150 out of 754 in Eigen-GARCH forecasting
151 out of 754 in Eigen-GARCH forecasting
152 out of 754 in Eigen-GARCH forecasting
153 out of 754 in Eigen-GARCH forecasting
154 out of 754 in Eigen-GARCH forecasting
155 out of 754 in Eigen-GARCH forecasting


/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:44: RuntimeWarning: overflow encountered in true_divide
  lls3 = np.log(lambdas[2,:]) + np.multiply(Xtilde[2,:],Xtilde[2,:]) / lambdas[2,:]


156 out of 754 in Eigen-GARCH forecasting
157 out of 754 in Eigen-GARCH forecasting
158 out of 754 in Eigen-GARCH forecasting
159 out of 754 in Eigen-GARCH forecasting
160 out of 754 in Eigen-GARCH forecasting
161 out of 754 in Eigen-GARCH forecasting
162 out of 754 in Eigen-GARCH forecasting
163 out of 754 in Eigen-GARCH forecasting
164 out of 754 in Eigen-GARCH forecasting
165 out of 754 in Eigen-GARCH forecasting
166 out of 754 in Eigen-GARCH forecasting


/Users/silkesommerfugl/Desktop/Master/Eigen_GARCH.py:42: RuntimeWarning: overflow encountered in true_divide
  lls1 = np.log(lambdas[0,:]) + np.multiply(Xtilde[0,:],Xtilde[0,:]) / lambdas[0,:]


167 out of 754 in Eigen-GARCH forecasting
168 out of 754 in Eigen-GARCH forecasting
169 out of 754 in Eigen-GARCH forecasting
170 out of 754 in Eigen-GARCH forecasting
171 out of 754 in Eigen-GARCH forecasting
172 out of 754 in Eigen-GARCH forecasting
173 out of 754 in Eigen-GARCH forecasting
174 out of 754 in Eigen-GARCH forecasting
175 out of 754 in Eigen-GARCH forecasting
176 out of 754 in Eigen-GARCH forecasting
177 out of 754 in Eigen-GARCH forecasting
178 out of 754 in Eigen-GARCH forecasting
179 out of 754 in Eigen-GARCH forecasting
180 out of 754 in Eigen-GARCH forecasting
181 out of 754 in Eigen-GARCH forecasting
182 out of 754 in Eigen-GARCH forecasting
183 out of 754 in Eigen-GARCH forecasting
184 out of 754 in Eigen-GARCH forecasting
185 out of 754 in Eigen-GARCH forecasting
186 out of 754 in Eigen-GARCH forecasting
187 out of 754 in Eigen-GARCH forecasting
188 out of 754 in Eigen-GARCH forecasting
189 out of 754 in Eigen-GARCH forecasting
190 out of 754 in Eigen-GARCH fore

In [37]:
# Make a list for each of the covariance pairs
cov_JPM_KO_forcast = []
for t in range(1,N):
    cov_JPM_KO_forcast.append(forecastMatrix_eig[t][0][1])

cov_JPM_MSFT_forcast = []
for t in range(1,N):
    cov_JPM_MSFT_forcast.append(forecastMatrix_eig[t][0][2])

cov_KO_MSFT_forcast = []
for t in range(1,N):
    cov_KO_MSFT_forcast.append(forecastMatrix_eig[t][1][2])

In [40]:
df = pd.DataFrame(cov_JPM_KO_forcast )
df.to_csv('cov_JPM_KO_forecast_eigen.csv') 

df = pd.DataFrame(cov_JPM_MSFT_forcast )
df.to_csv('cov_JPM_MSFT_forcast_eigen.csv') 

df = pd.DataFrame(cov_KO_MSFT_forcast )
df.to_csv('cov_KO_MSFT_forcast_eigen.csv') 


In [32]:
# save variances
var_BAC_forcast = []
for t in range(1,N):
    var_BAC_forcast.append(forecastMatrix_eig[t][0][0])

var_JPM_forcast = []
for t in range(1,N):
    var_JPM_forcast.append(forecastMatrix_eig[t][1][1])

var_WFC_forcast = []
for t in range(1,N):
    var_WFC_forcast.append(forecastMatrix_eig[t][2][2])

df = pd.DataFrame(var_BAC_forcast)
df.to_csv('var_BAC_forcast_eigen.csv') 

df = pd.DataFrame(var_JPM_forcast)
df.to_csv('var_JPM_forcast_eigen.csv') 

df = pd.DataFrame(var_WFC_forcast)
df.to_csv('var_WFC_forcast.csv') 

## MVP

In [34]:
weights_eigen = np.zeros((N,1,p))
min_port_ret = np.zeros((N,1))

ell = np.array(([1, 1, 1]))
for n in range(1,N):
    w = np.matmul(np.linalg.inv(forecastMatrix_eig[n]), ell)
    weights_eigen[n,:,:] = w/sum(w)
    min_port_ret[n] = np.sum((np.multiply(weights_eigen[n,:,:], data_forecast[:,1486+n])),axis=1)
    

In [55]:
df = pd.DataFrame((min_port_ret))
df.to_csv('MVP_eigen.csv') 

df = pd.DataFrame((min_port_ret_real))
df.to_csv('MVP_real.csv')

In [53]:
mean_min = np.mean(min_port_ret)*np.sqrt(250)
std_min = np.std(min_port_ret)*np.sqrt(250)
sharpe_ratio_min = mean_min/std_min
print(mean_min, std_min, sharpe_ratio_min)

0.7875789811831866 24.588563019328742 0.032030297198094956
